<img src="https://github.com/christopherhuntley/BUAN6510/blob/master/img/Dolan.png?raw=true" width="180px" align="right">

# **BUAN 6510**
# **Lesson 2: Basic SELECT Statements** 
_Retrieving data from a single table._

## **Learning Objectives**
### **Theory / Be able to explain ...**
- The sequence and purpose of each clause of SQL select queries (SELECT, FROM, WHERE, GROUP BY, etc.)
- How SELECT queries compare to similar operations in Excel or Pandas 

### **Skills / Know how to ...**
- Write basic SQL SELECT / FROM / WHERE queries
- Apply functions and conditionals where required
- Calculate aggregate quantities like AVG, SUM, etc.
- Group records using column selectors

--------
## **LESSON 2 HIGHLIGHTS**

In [ ]:
#@title Run this cell if video is does not appear TODO REPLACE WITH NEW VIDEO
%%html
<div style="max-width:1000px">
  <div style="position: relative;padding-bottom: 56.25%;height: 0;">
    <iframe style="position: absolute;top: 0;left: 0;width: 100%;height: 100%;" rel="0" modestbranding="1"  src="https://www.youtube.com/embed/YkDLv6CtEnc" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
  </div>
</div>

## **BIG PICTURE: SQL as a universal data access language** 

Among programming skills, coding in SQL is just not like the others. Consider, for example, the January 2021 *Tiobe Index* of popular programming languages, where SQL is listed as #12. Every other language on the list can be used to build apps and systems. We call them general purpose languages. SQL, however, is explicitly a special purpose language, designed for data management and only data management. Of the others on the list the most similar in that respect is R. Though it is certainly possible to create apps in R, the focus is on data analysis and only data analysis. Everything else is strictly general purpose.  So, if SQL is so out of step with the rest of the programming language universe, why is it considered a critical data analytics skill? Because it does its job extremely well, of course, but there is more to it than that. 

![Tiobe Index](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L2_tiobe_index.png)

SQL is an ancient programming language. Of the others on the list, the only one that could be considered a contemporary is C. (Assembly Language is technically not a single language but a kind of language, so it does not have an age *per se*.) Both C and SQL were developed in the late 1960s and released in the early 1970s. The others came along *decades* later. By then C and SQL were already ubiquitous, in use by millions of programmers around the world. 

Age and "first mover advantage" are not all of the SQL story though. Recall the three tiered model from Lesson 1? Each of the tiers are treated by the others as a black box, with internal implementation details hidden behind a *public interface* (often called an API). As long as a given tier can handle certain requests, phrased in such and such a way and producing results in specified format, then nobody has to care *how* it works. That is extremely powerful! SQL just happens to provide just such an interface. For basic CRUD operations in the data tier, SQL provides a complete and standard set of requests and responses that any application or system can rely on. Further, if a better implementation of the standards embodied by SQL comes along, we can upgrade to the newer technology without modifying *any* existing code. 

So, when asked "Why SQL?" the best answer is "Why not SQL?" because that is the right way to think of it. SQL works so well for many, many applications supported by millions of programmers. Thus, the burden of proof for an application programmer to disqualify SQL is to show that SQL can't get the job done and then propose something that does. Even then, the smart boss will instinctively ask for the same data integrity protections provided by SQL. In the end, using anything else is like swimming upstream, where relatively gentle currents on a clear day can become deadly in a storm. In other words, never assume that what works for local storage on an isolated iPhone will work for the cloud servers it connects with. The volume of data and the likelihood of corruption is just too high. Some companies, usually startups, that made the wrong choice are having to live with their decisions and ... scaling up to SQL compliance. Why not start there from the beginning? 


 








## **SQL History, Standards, and Use Cases**


## **SQL SELECT Statements ... one clause at a time**

### **SELECT**

### **FROM**

### **WHERE**

### **GROUP BY**

### **HAVING**

### **ORDER BY**

### **LIMIT**



## **Aliases and Views**

### **Column Aliases**

### **Expression Aliases**

### **Views (Query Aliases)**

## **Boolean Expressions and WHERE Clauses**

## **Complex Calculations with SQL Functions and CASE Expressions**

## **Grouping and Aggregation**

## **PRO TIPS: How to write queries that work the first time, every time.**

## **SQL AND BEYOND: MySQL DBMS**

## **Congratulations! You've made it to the end of Lesson 2.**

In Lesson 3 we will continue on to more advanced SELECT statements that involve multiple tables or subqueries. That's about as far as most data scientists go ... but we will of course continue on with even more advanced topics. There is *a lot more* to SQL than just retrieving data. 